# MTH 373 - Homework #2

**Name:** <font color="red">**_Hai T. Nguyen_**</font>

**Due:** Monday, April 29, 2024 (submit on Canvas before 11:30 am)

In [1]:
import numpy as np
import scipy.sparse as sp

### Preliminaries

We will be using the `load_as_csr`, `num_rows`, and `matvec` functions from homework #1. They are provided below for you to use.

In [2]:
def load_as_csr(filename):
    # Read m, n, and nnz from the file
    m, n, nnz = np.genfromtxt(filename, max_rows=1, dtype=None)
    # Read the data as a list of tuples [(i,j,v), ...]
    data = np.genfromtxt(filename, skip_header=1, dtype=None)
    # Allocate arrays
    I = np.zeros(m + 1, dtype=int)
    J = np.zeros(nnz, dtype=int)
    V = np.zeros(nnz)
    
    for k in range(nnz):
        i = data[k][0]
        I[i+1] += 1
        
    for i in range(m):
        I[i+1] += I[i]
    
    for k in range(nnz):
        i, j, v = data[k]
        idx = I[i]
        I[i] += 1
        J[idx] = j
        V[idx] = v
    
    for i in reversed(range(m)):
        I[i+1] = I[i]
        
    I[0] = 0
    
    return I, J, V

def num_rows(A):
    return len(A[0]) - 1

def matvec(A, v):
    I, J, V = A
    m = num_rows(A)
    w = np.zeros(m)
    
    for i in range(m):
        begin = I[i]
        end = I[i+1]
        for idx in range(begin, end):
            j = J[idx]
            w[i] += V[idx]*v[j]
    
    return w

### 1. Sparse triangular solves (5 points)

Write functions `tril_solve` and `triu_solve` that take a matrix $A$ in CSR format (as a tuple of three arrays, the output of `load_as_csr`) and a right-hand side vector $b$, and returns the solution to $L x = b$ and $U x = b$, where $L$ and $U$ are the lower and upper triangular parts of $A$, respectively.

In [3]:
def tril_solve(A, b):
    I, J, V = A
    n = len(b)
    x = np.zeros(n)

    # Construct lower triangular matrix L from A
    L = np.zeros((n, n))
    for i in range(n):
        for jj in range(I[i], I[i+1]):
            j = J[jj]
            if i >= j:  # Only fill lower triangular part
                L[i][j] = V[jj]

    # Perform lower-triangular solve (forward substitution)
    for i in range(n):
        x[i] = b[i]
        for j in range(i):
            x[i] = x[i] - (L[i][j] * x[j])
        x[i] = x[i] / L[i][i]
    return x

def triu_solve(A, b):
    I, J, V = A
    n = len(b)
    x = np.zeros(n)

    # Construct upper triangular matrix U from A
    U = np.zeros((n, n))
    for i in range(n):
        for jj in range(I[i], I[i+1]):
            j = J[jj]
            if i <= j:  # Only fill upper triangular part
                U[i][j] = V[jj]

    # Perform upper-triangular solve (backward substitution)
    for i in range(n-1, -1, -1):
        x[i] = b[i]
        for j in range(i+1, n):
            x[i] = x[i] - (U[i][j] * x[j])
        x[i] = x[i] / U[i][i]
    return x

The code below can be used to check your results.

In [4]:
def to_scipy(A):
    return sp.csr_matrix((A[2], A[1], A[0]))

def triu_solve_scipy(A, b):
    return sp.linalg.spsolve(sp.triu(to_scipy(A), format="csr"), b)

def tril_solve_scipy(A, b):
    return sp.linalg.spsolve(sp.tril(to_scipy(A), format="csr"), b)

for filename in ["16.txt", "25.txt", "50.txt", "64.txt"]:
    A = load_as_csr(filename)
    n = num_rows(A)
    b = np.random.default_rng(0).random(n)
    x_l_1 = tril_solve(A, b)
    x_l_2 = tril_solve_scipy(A, b)

    if np.linalg.norm(x_l_1 - x_l_2) < 1e-10:
        print("tril_solve OK")
    else:
        print("tril_solve incorrect")

    x_u_1 = triu_solve(A, b)
    x_u_2 = triu_solve_scipy(A, b)

    if np.linalg.norm(x_u_1 - x_u_2) < 1e-10:
        print("triu_solve OK")
    else:
        print("triu_solve incorrect")

tril_solve OK
triu_solve OK
tril_solve OK
triu_solve OK
tril_solve OK
triu_solve OK
tril_solve OK
triu_solve OK


### 1b. (2 points)

What is the computational complexity of the above functions (in terms of the size of the matrix and the number of nonzeros)? Justify your answer.

### 2. Gauss-Seidel method (5 points)

Write a function `gauss_seidel` that takes a CSR matrix $A$, right-hand side $b$, and implements the Gauss-Seidel method. The string parameter `uplo` will determine if the lower or upper part of the matrix will be used: a value of `"L"` indicates the lower triangular part, and `"U"` indicates the upper triangular part. Use the functions `matvec`, `tril_solve`, and `triu_solve` from above.

Perform a maximum of 1000 iterations, and terminate when the norm of the residual (computed using `np.linalg.norm`) is below `1e-5`.

When the method converges, return the solution, and print the message `"Converged after {i} iterations"`, where `{i}` denotes the number of iterations.

Use the code below to run Gauss-Seidel and output the number of iterations.

In [5]:
def gauss_seidel(A, b, uplo="L"):
    I, J, V = A
    n = len(b)
    x = np.zeros(n)
    max_iterations = 1000
    tolerance = 1e-5
    for k in range(max_iterations):
        r = b - matvec(A,x)
        residual = np.linalg.norm(r)
        if residual < tolerance:
            print(f"Converged after {k+1} iterations")
            return x
        if (uplo == "L"):
            x = x + tril_solve(A, r)
        elif (uplo == "U"):
            x = x + triu_solve(A, r)
    print("Did not converge within maximum iterations")
    return x

In [6]:
for filename in ["16.txt", "25.txt", "50.txt", "64.txt"]:
    A = load_as_csr(filename)
    b = np.random.default_rng(0).random(num_rows(A))
    gauss_seidel(A, b)

Converged after 193 iterations
Converged after 64 iterations
Converged after 99 iterations
Converged after 106 iterations


### 3. Modify the matrices and check convergence (5 points)

Write a function `add_to_diagonal` that takes a CSR matrix $A$ (the output of `load_as_csr`), and returns a **new** matrix, that has the value `alpha` added to the diagonal entries, i.e. $A + \alpha I$.

Re-run the tests above for matrices $A + \alpha I$, with $\alpha \in \{0, 1, 2, 4, 8\}$. Output iteration counts for each test matrix, and for each value of $\alpha$.

What happens for larger $\alpha$? Can you explain this in terms of the eigenvalues of the matrix?

In [7]:
def add_to_diagonal(A, alpha):
    I, J, V = A
    n = num_rows(A)
    V2 = np.copy(V)
    # Modify the diagonal entries by adding alpha
    for i in range(n):
        begin = I[i]
        end = I[i+1]
        for idx in range(begin, end):
            j = J[idx]
            if ( j == i):
                V2[j] = V2[j] + alpha
    return I, J, V2

In [8]:
alpha = [0, 1, 2, 4, 8]
for i in range(len(alpha)- 1):
    A = add_to_diagonal(A, alpha[i])
    print(A)
    print()

(array([  0,   6,  13,  19,  25,  31,  37,  43,  48,  54,  60,  68,  75,
        82,  87,  93,  99, 103, 106, 114, 122, 130, 134, 140, 146, 151,
       158, 161, 168, 170, 176, 183, 188, 193, 199, 203, 207, 211, 216,
       220, 224, 231, 238, 245, 248, 254, 260, 267, 272, 278, 285, 294,
       298, 303, 311, 318, 326, 333, 336, 340, 344, 350, 355, 360, 366]), array([16, 13, 41,  0,  4, 19, 53, 14,  5, 52,  1, 22, 62, 55, 37, 47,  2,
       49, 30,  9, 18, 11, 20,  3, 45, 16,  0, 33, 19,  4, 40, 53, 41, 13,
        5,  1, 52, 46, 48, 10, 23,  7,  6, 48,  6, 23, 27,  7, 56, 46, 55,
       17, 39,  8, 18,  3, 25, 32, 45,  9, 55, 46,  6, 23, 50, 30, 54, 10,
        3, 20, 42, 18, 11, 44, 50, 31, 24, 42, 63, 27, 58, 12, 16,  5,  0,
       41, 13, 53, 49, 21, 22,  1, 14, 33, 29, 60, 15, 35, 38,  4,  0, 16,
       13, 56,  8, 17,  9,  3, 25, 11, 50, 40, 18, 19,  0, 50, 54, 41,  4,
       40, 18, 19,  3, 11, 51, 45, 42, 61, 63, 20, 49, 14, 47, 21, 36, 14,
        1, 62, 34, 22,  6, 10, 50, 23